In [1]:
!pip install sqlalchemy -q


[notice] A new release of pip is available: 23.1.2 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


# Part 1: SQLAlchemy Core

# 1. Setup

In [7]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, ForeignKey, DateTime, text, select, insert, update, delete, func, join
from datetime import datetime

In [9]:
# Create an in-memory SQLite database (echo=True - for learning purposes)
engine = create_engine("sqlite:///:memory:", echo=True)

In [10]:
# Metadata is a container for Table objects
metadata = MetaData()

In [11]:
print("Engine created:", engine)

Engine created: Engine(sqlite:///:memory:)


# 2. Define tables

In [13]:
# Define authors table
authors = Table(
    'authors', 
    metadata, 
    Column('id', Integer, primary_key=True),
    Column('name', String(100), nullable=False),
    Column('email', String(200), unique=True)
)

# Define articles table with foreign key
articles = Table(
    'articles', 
    metadata,
    Column('id', Integer, primary_key=True),
    Column('title', String(200), nullable=False),
    Column('content', String(5000)),
    Column('author_id', Integer, ForeignKey('authors.id')),
    Column('created_at', DateTime, default=datetime.utcnow)
)

In [14]:
# Create tables in database
metadata.create_all(engine)

2026-02-24 17:26:27,061 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-24 17:26:27,066 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("authors")
2026-02-24 17:26:27,068 INFO sqlalchemy.engine.Engine [raw sql] ()
2026-02-24 17:26:27,069 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("authors")
2026-02-24 17:26:27,071 INFO sqlalchemy.engine.Engine [raw sql] ()
2026-02-24 17:26:27,072 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("articles")
2026-02-24 17:26:27,073 INFO sqlalchemy.engine.Engine [raw sql] ()
2026-02-24 17:26:27,073 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("articles")
2026-02-24 17:26:27,074 INFO sqlalchemy.engine.Engine [raw sql] ()
2026-02-24 17:26:27,075 INFO sqlalchemy.engine.Engine 
CREATE TABLE authors (
	id INTEGER NOT NULL, 
	name VARCHAR(100) NOT NULL, 
	email VARCHAR(200), 
	PRIMARY KEY (id), 
	UNIQUE (email)
)


2026-02-24 17:26:27,075 INFO sqlalchemy.engine.Engine [no key 0.00068s] ()
2026-02-24 17:26:27,076 INFO sqlalchemy

In [27]:
print("Tables created!")
print(f"Columns in authors: {[c.name for c in authors.columns]}")
print(f"Columns in articles: {[c.name for c in articles.columns]}")

Tables created!
Columns in authors: ['id', 'name', 'email']
Columns in articles: ['id', 'title', 'content', 'author_id', 'created_at']


# 3. Insert Data

In [16]:
# Single insert
with engine.begin() as conn: 
    result = conn.execute(
        insert(authors).values(name="Alice Johnson", email = "alice@example.com")
    )
    print(f"Inserted author ID: {result.inserted_primary_key}")

2026-02-24 17:29:03,287 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-24 17:29:03,290 INFO sqlalchemy.engine.Engine INSERT INTO authors (name, email) VALUES (?, ?)
2026-02-24 17:29:03,291 INFO sqlalchemy.engine.Engine [generated in 0.00119s] ('Alice Johnson', 'alice@example.com')
Inserted author ID: (1,)
2026-02-24 17:29:03,292 INFO sqlalchemy.engine.Engine COMMIT


In [23]:
# Bulk insert 
with engine.begin() as conn: 
    articles_data = [
        {"title": "Frist Post", "content": "Hello world!", "author_id": 1},
        {"title": "Second Post", "content": "More content", "author_id": 1},
        {"title": "Bob's Article", "content": "Writing from Bob.", "author_id": 2},
    ]
    result = conn.execute(insert(articles), articles_data)
    print(f"Inserted {result.rowcount} articles")

Inserted 3 articles


# 4. Select Data

In [24]:
# Select all authors
engine.echo = False # Turn off echo for cleaner output
with engine.connect() as conn: 
    result = conn.execute(select(authors))
    for row in result: 
        print(f"ID: {row.id}, Name: {row.name}, Email: {row.email}")

ID: 1, Name: Alice Johnson, Email: alice@example.com


In [28]:
# Select withh WHERE clause
with engine.connect() as conn:
    stmt = select(articles).where(articles.c.author_id == 1)
    result = conn.execute(stmt)
    for row in result:
        print(row)
        print(f"{row.title}: {row.content[:30]}...")

(1, 'Frist Post', None, 1, datetime.datetime(2026, 2, 26, 15, 19, 46, 624619))


TypeError: 'NoneType' object is not subscriptable

# I did not insert bob and re-ran cells hence the error
Perfect opportunity to learn

In [34]:
# Clearning existing data
with engine.begin() as conn: 
    conn.execute(delete(articles))
    conn.execute(delete(authors))
    print("Cleared existing data")

Cleared existing data


In [38]:
# Creating authors first
with engine.begin() as conn: 
    conn.execute(insert(authors).values(name = "Kriti Sanon", email="kritirobot@example.com" ))
    conn.execute(insert(authors).values(name = "Ananya Pandey", email="anayachunky@example.com"))
    print("Inserted 2 authors")

Inserted 2 authors


In [42]:
with engine.begin() as conn:
    articles_data = [
      {"title": "First Post", "content": "Hello world!", "author_id": 1},
      {"title": "Second Post", "content": "More content", "author_id": 1},
      {"title": "Pandey's Article", "content": "Writing from Ananya.", "author_id": 2},
      ]
    result = conn.execute(insert(articles), articles_data)
    print(f"Inserted {result.rowcount} articles")
    

Inserted 3 articles


In [45]:
engine.echo = False  # Turn off echo for cleaner output
with engine.connect() as conn:
  result = conn.execute(select(authors))
  for row in result:
      print(f"ID: {row.id}, Name: {row.name}, Email: {row.email}")

ID: 1, Name: Kriti Sanon, Email: kritirobot@example.com
ID: 2, Name: Ananya Pandey, Email: anayachunky@example.com


In [43]:
with engine.connect() as conn:
    stmt = select(articles).where(articles.c.author_id == 1)
    result = conn.execute(stmt)
    for row in result:
        print(f"{row.title}: {row.content[:30]}...")

First Post: Hello world!...
Second Post: More content...
